In [5]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


We want to create a dataframe to store the comments and their sentiment.
- 0: positive
- 1: neutral
-  2: negative
Since we are labeling the sentiment by hand, we want to get the rating in each comment to simplifying the process of categorizing.

In [116]:
import pandas as pd
df = pd.DataFrame(columns=['comment','rating','sentiment'])

## IMDB Scraping
 Now I want to scrape the IMDB website for movies and their reviews. First, I will scrape some imdb URLs to collect the movie ids.

In [137]:
import requests
from bs4 import BeautifulSoup
import re
import json

def get_movie_id(list, url):
    # URL for the IMDb top 250 movies
    movie_id_list = list
    url = url
    # Set headers to simulate a browser visit
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }
    
    # Send an HTTP request to get the page content
    response = requests.get(url, headers=headers)
    print(response.status_code)
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page: {response.status_code}")
        return []
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'lxml')
    

    # Find the title section of each movie which contains the link to the movie
    movies = soup.find_all("a", class_ = "ipc-title-link-wrapper")
    print(len(movies))
    # Collect the id of each movie by extracting it from the link
    
    for link in movies:
        href = link.get('href')  # Get the href attribute
        movie_id = href.split('/')[2]
        if not movie_id in movie_id_list:
            movie_id_list.append(movie_id)

    return movie_id_list


In [138]:
urls = ["https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_special,video&interests=in0000152",
    "https://www.imdb.com/list/ls542085053/?ref_=int_int_ls_slasher_i",
    "https://www.imdb.com/search/title/?title_type=feature,tv_movie&interests=in0000012",
    "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_special,video&interests=in0000034"]
movie_id_list = []
for url in urls:
    get_movie_id(movie_id_list, url)

print(movie_id_list)

200
25
200
20
200
25
200
25
['tt28607951', 'tt27131358', 'tt1262426', 'tt20194882', 'tt10655524', 'tt16426418', 'tt0120791', 'tt0103874', 'tt29892095', 'tt3774694', 'tt27419292', 'tt14230458', 'tt1684562', 'tt22741760', 'tt26047818', 'tt0120338', 'tt0121164', 'tt20215968', 'tt26446278', 'tt1099212', 'tt0109830', 'tt0099487', 'tt0338013', 'tt1340094', 'tt2322441', 'tt0094035', 'tt0075342', 'tt6535880', 'tt0093996', 'tt6426028', 'tt0082748', 'tt0082118', 'tt0085253', 'tt0095379', 'tt0422401', 'tt5022702', 'tt8998472', 'tt0102690', 'tt0076150', 'tt0070791', 'tt0113376', 'tt0093677', 'tt0088117', 'tt2190838', 'tt0083542', 'tt16366836', 'tt9218128', 'tt6263850', 'tt8864596', 'tt7766378', 'tt14948432', 'tt22022452', 'tt1270797', 'tt0172495', 'tt29195603', 'tt12584954', 'tt0816692', 'tt15239678', 'tt0241527', 'tt17279496', 'tt12037194', 'tt4919268', 'tt0120737', 'tt5950044', 'tt21235248', 'tt33998607', 'tt4978420', 'tt1375666', 'tt6587046', 'tt0338526', 'tt2049403', 'tt9860566', 'tt0094721', 

In [ ]:
%pip install selenium

In [ ]:
%pip install selenium==4.4.3

In [ ]:
%pip uninstall chromedriver-py

Next, I will visit each movie page with the retrieved ids and scrape the reviews and ratings.

In [157]:
def get_imdb_reviews(list,movie_id):
    # URL for the IMDb user reviews page
    url = f"https://www.imdb.com/title/{movie_id}/reviews?ref_=tt_ql_3"
    reviews_list = list
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }
    
    # Send an HTTP request to get the page content
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'html.parser')


    # Find the review containers
    review_containers = soup.find_all('div', class_='ipc-list-card__content')  # Adjust this class if needed

    for container in review_containers:
        review = container.find('div', class_='ipc-html-content-inner-div')
        rating = container.find('span', class_='ipc-rating-star')
        if review:
            obj = {"comment": review.text,
                "rating": int(rating.text.split("/")[0]) if rating else None }
            
            reviews_list.append(obj)
  
    return reviews_list


In [159]:
# Collect the text of each review
reviews_list = []
for movie_id in movie_id_list:
    get_imdb_reviews(reviews_list,movie_id)

print(len(reviews_list))

1511


In [160]:
#add the data to the dataframe
df = pd.DataFrame(reviews_list)
df["sentiment"] = df["rating"].apply(lambda x: "neutral" if x == 5 else ("negative" if x < 5 else ( "positive" if x > 5 else None)))

try:
    df.to_csv("movie_reviews.csv", index=False)  # index=False if you don’t need row indices
    print("File saved successfully.")
except Exception as e:
    print("Error:", e)

File saved successfully.


In [161]:
df.head()

,comment,rating,sentiment
0,I just watched one of the best films of the ye...,8.0,positive
1,Watching this film at the Cannes Film Festival...,9.0,positive
2,"The hype surrounding Anora is colossal, and it...",9.0,positive
3,I am not really familiar with Sean Baker as a ...,10.0,positive
4,Sean Baker returns to his wheelhouse of explor...,10.0,positive


In [164]:
df.describe()

,rating
count,1411.000000
mean,6.722892
std,2.440301
min,1.000000
25%,6.000000
50%,7.000000
75%,8.000000
max,10.000000
